In [1]:
from ipywidgets import interact, widgets
interact(lambda x: x**2, x=widgets.IntSlider(min=0, max=10));

interactive(children=(IntSlider(value=0, description='x', max=10), Output()), _dom_classes=('widget-interact',…

# 📈 The Solow Growth Model

The Solow growth model, developed by Robert Solow, is a foundational model in macroeconomics used to explain long-run economic growth. It describes how capital accumulation, population growth, and technological progress interact to determine an economy's output level and growth path over time.

The model uses a standard aggregate production function, typically Cobb-Douglas with constant returns to scale:
$$Y = K^{\alpha} (AL)^{1 - \alpha}$$
- $Y$: Total output
- $K$: Physical capital stock
- $L$: Labor force
- $A$: Level of technology (labor-augmenting)
- $\alpha$: Capital's share of income ($0 < \alpha < 1$)

The term $AL$ represents **effective labor**. The model analyzes the economy's dynamics by focusing on capital accumulation **per effective worker**.

# ⚙️ Dynamics in Intensive Form & Steady State

To analyze convergence, we look at variables per effective worker:
- $k = K / (AL)$: Capital per effective worker
- $y = Y / (AL)$: Output per effective worker ($y = k^{\alpha}$)

The core dynamic equation describes the change in capital per effective worker ($\dot{k}$ or $\Delta k$):
$$ \dot{k} = s y - (\delta + n + g) k $$
$$ \dot{k} = s k^{\alpha} - (\delta + n + g) k $$
*(Discrete time approx: $k_{t+1} - k_t \approx s k_t^{\alpha} - (\delta + n + g) k_t$)*

Where:
- $s$: Savings rate (fraction of output invested)
- $\delta$: Depreciation rate of capital
- $n$: Growth rate of the labor force ($L$)
- $g$: Growth rate of technology ($A$)

**Interpretation:**
- $s k^{\alpha}$: Actual investment per effective worker.
- $(\delta + n + g) k$: "Breakeven investment" per effective worker – the amount needed to keep $k$ constant. Capital must be replaced due to depreciation ($\delta k$), equipped for new workers ($nk$), and keep up with technology ($gk$).

**Steady State ($k^*$):** The economy converges to a steady state where $k$ is constant ($\dot{k} = 0$). This occurs when actual investment equals breakeven investment:
$$ s (k^*)^{\alpha} = (\delta + n + g) k^* $$
Solving for $k^*$:
$$ k^* = \left( \frac{s}{\delta + n + g} \right)^{\frac{1}{1-\alpha}} $$
The steady-state output per effective worker is $y^* = (k^*)^{\alpha}$.

**Special Case (No Growth in A or L):** If $n=0$ and $g=0$, the model describes capital per *worker* ($K/L$) converging to $k^* = (sA / \delta)^{1/(1-\alpha)}$. In this case, output per worker ($Y/L$) also converges to a constant steady-state level, and there is no long-run growth in living standards.

# 📊 Balanced Growth Path (BGP)

When $g > 0$ (positive technological progress), the economy reaches a **Balanced Growth Path (BGP)**, characterized by:

1.  **Constant $k$ and $y$:** Capital per *effective* worker ($k=K/AL$) and output per *effective* worker ($y=Y/AL$) converge to their constant steady-state values ($k^*, y^*$).
2.  **Constant Growth in Per Capita Variables:** Capital per *worker* ($K/L = k \times A$) and output per *worker* ($Y/L = y \times A$) grow at the rate of technological progress, $g$. This implies sustained growth in living standards.
3.  **Constant Growth in Aggregate Variables:** Total capital ($K = k \times A \times L$) and total output ($Y = y \times A \times L$) grow at the rate $n+g$.
4.  **Constant Capital-Output Ratio:** The ratio $K/Y = k/y = (k^*)^{1-\alpha}$ is constant in the steady state.

The key prediction is that **sustained long-run growth in output per worker (living standards) is driven solely by technological progress ($g$)**. Changes in the savings rate ($s$) or population growth ($n$) affect the *level* of the BGP ($k^*$ and $y^*$) but not its long-run *growth rate*.

The simulation below plots the transition of $k$ and $y$ towards their steady states, and also shows the growth path of output per worker ($Y/L$) on the BGP.

In [2]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider, Layout
from IPython.display import display, Markdown
import warnings

# Optional: Use a specific style
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except IOError:
    pass # Use default if style not found

def solow_growth_simulator(
    s=0.25, delta=0.05, alpha=0.33, n=0.01, g=0.02, A0=1.0, k0=1.0, T=150
    ):
    """
    Simulates the Solow Growth Model with population growth (n) and
    technological progress (g), showing convergence to the Balanced Growth Path (BGP).

    Args:
        s (float): Savings rate.
        delta (float): Depreciation rate.
        alpha (float): Capital share (output elasticity of capital).
        n (float): Population / Labor force growth rate.
        g (float): Technological progress growth rate (labor-augmenting).
        A0 (float): Initial level of technology.
        k0 (float): Initial capital per effective worker (K0 / (A0*L0)).
        T (int): Time Horizon (number of periods).
    """
    # Ensure T is an integer
    T = int(T)
    if T < 5: T = 5

    # Input validation
    alpha = np.clip(alpha, 0.01, 0.99)
    s = np.clip(s, 0.01, 0.99)
    delta = max(delta, 0)
    n = max(n, 0)
    g = max(g, 0)
    k0 = max(k0, 0.01) # Start k slightly above zero
    A0 = max(A0, 0.1)

    # Calculate steady state (per effective worker)
    break_even_rate = delta + n + g
    k_star = np.nan
    y_star = np.nan
    if break_even_rate > 1e-9 and s > 0 : # Ensure valid calculation
        try:
            # k* = (s / (delta + n + g))**(1 / (1 - alpha))
            exponent = 1.0 / (1.0 - alpha)
            base = s / break_even_rate
            log_k_star = exponent * np.log(base)
            # Set a reasonable upper bound to prevent extreme values in plots
            log_k_star = min(log_k_star, 20) # Corresponds to k_star ~ 5e8
            k_star = np.exp(log_k_star)
            y_star = k_star**alpha
        except (ValueError, OverflowError):
            warnings.warn("Could not calculate steady state - parameters might lead to extreme values.")
            k_star, y_star = np.nan, np.nan


    # --- Simulation ---
    k = np.zeros(T + 1) # Capital per effective worker K/(AL)
    A = A0 * (1 + g)**np.arange(T + 1) # Technology level
    # L is not explicitly needed for k dynamics, but needed for Y/L

    k[0] = k0
    for t in range(T):
        y_t = k[t]**alpha # Output per effective worker
        investment = s * y_t
        break_even_investment = break_even_rate * k[t]
        # Change in k = investment - break_even_investment
        delta_k = investment - break_even_investment
        k[t+1] = k[t] + delta_k
        # Ensure k doesn't go below a small positive value
        k[t+1] = max(k[t+1], 1e-9)

    # Calculate derived variables
    y = k**alpha # Output per effective worker y = Y/(AL)
    y_per_worker = y * A # Output per worker Y/L = y * A

    # --- Plotting ---
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    time = np.arange(T + 1)

    # Plot 1: Per Effective Worker Variables (Convergence to Steady State)
    axes[0].plot(time, k, label='k(t) = K/(AL)', color='navy', lw=2)
    axes[0].plot(time, y, label='y(t) = Y/(AL)', color='forestgreen', lw=2, linestyle='--')
    if not np.isnan(k_star):
        axes[0].axhline(k_star, color='red', linestyle=':', lw=1.5, label=f'Steady State k* ≈ {k_star:.2f}')
    if not np.isnan(y_star):
        axes[0].axhline(y_star, color='orange', linestyle=':', lw=1.5, label=f'Steady State y* ≈ {y_star:.2f}')
    axes[0].set_title("Convergence per Effective Worker")
    axes[0].set_xlabel("Time (t)")
    axes[0].set_ylabel("Level per Effective Worker")
    axes[0].legend()
    axes[0].grid(True, linestyle='--', alpha=0.7)
    # Dynamic Y limits for k/y plot
    k_plot_max = max(k.max(), k_star if not np.isnan(k_star) else k.max())
    y_plot_max = max(y.max(), y_star if not np.isnan(y_star) else y.max())
    axes[0].set_ylim(0, max(k_plot_max, y_plot_max) * 1.1)
    axes[0].set_xlim(0, T)


    # Plot 2: Per Worker Variables (Balanced Growth Path - Log Scale)
    axes[1].plot(time, y_per_worker, label='Output per Worker (Y/L = y*A)', color='purple', lw=2)
    # Optional: Plot K/L = k*A
    # k_per_worker = k * A
    # axes[1].plot(time, k_per_worker, label='Capital per Worker (K/L = k*A)', color='deepskyblue', lw=2, linestyle='--')
    axes[1].set_yscale('log') # Log scale shows constant growth as straight line
    axes[1].set_title(f"Balanced Growth Path (Log Scale, Growth ≈ g={g:.1%})")
    axes[1].set_xlabel("Time (t)")
    axes[1].set_ylabel("Level per Worker (Log Scale)")
    axes[1].legend()
    axes[1].grid(True, which='both', linestyle='--', alpha=0.7) # Grid on log scale
    axes[1].set_xlim(0, T)

    fig.suptitle(f"Solow Growth Model Simulation (s={s:.2f}, δ={delta:.2f}, n={n:.1%}, g={g:.1%}, α={alpha:.2f})", fontsize=14, y=1.02)
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

    # --- Display Steady State Info ---
    steady_state_md = f"""
    ### ⚖️ Steady State & BGP Summary:

    * **Steady-State Capital per Effective Worker (k\*):** {k_star:.3f}
    * **Steady-State Output per Effective Worker (y\*):** {y_star:.3f}
        * *(Variables per effective worker become constant)*
    * **BGP Growth Rate of Output per Worker (g(Y/L)):** {g:.2%}
    * **BGP Growth Rate of Total Output (gY):** {n + g:.2%}
        * *(Per worker variables grow at rate g, Total variables grow at rate n+g)*
    """
    display(Markdown(steady_state_md))


# --- Create Interactive Widgets ---
style = {'description_width': 'initial'}
layout = Layout(width='95%')

interact(
    solow_growth_simulator,
    s=FloatSlider(value=0.25, min=0.05, max=0.6, step=0.01, description='Savings Rate (s):', style=style, layout=layout, readout_format='.2f'),
    delta=FloatSlider(value=0.05, min=0.01, max=0.15, step=0.005, description='Depreciation (delta δ):', style=style, layout=layout, readout_format='.3f'),
    alpha=FloatSlider(value=0.33, min=0.1, max=0.8, step=0.01, description='Capital Share (alpha α):', style=style, layout=layout, readout_format='.2f'),
    n=FloatSlider(value=0.01, min=0.0, max=0.05, step=0.002, description='Pop. Growth (n):', style=style, layout=layout, readout_format='.1%'),
    g=FloatSlider(value=0.02, min=0.0, max=0.05, step=0.002, description='Tech Growth (g):', style=style, layout=layout, readout_format='.1%'),
    A0=FloatSlider(value=1.0, min=0.5, max=3.0, step=0.1, description='Initial Tech (A0):', style=style, layout=layout),
    k0=FloatSlider(value=1.0, min=0.1, max=20.0, step=0.1, description='Initial k (k0=K0/A0L0):', style=style, layout=layout, readout_format='.1f'),
    T=IntSlider(value=150, min=20, max=500, step=10, description='Time Periods (T):', style=style, layout=layout)
);


<>:136: SyntaxWarning: invalid escape sequence '\*'
<>:134: SyntaxWarning: invalid escape sequence '\*'
<>:136: SyntaxWarning: invalid escape sequence '\*'
<>:134: SyntaxWarning: invalid escape sequence '\*'
/var/folders/kz/cc2y6j3953z1bgkfgj5xk9yc0000gn/T/ipykernel_87520/2052404946.py:136: SyntaxWarning: invalid escape sequence '\*'
  
/var/folders/kz/cc2y6j3953z1bgkfgj5xk9yc0000gn/T/ipykernel_87520/2052404946.py:134: SyntaxWarning: invalid escape sequence '\*'
  display(Markdown(steady_state_md))


interactive(children=(FloatSlider(value=0.25, description='Savings Rate (s):', layout=Layout(width='95%'), max…